<a href="https://colab.research.google.com/github/heeramani/A-to-Z-Resources-for-Students/blob/master/V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
import numpy as np
import nltk
import re
import sys, os
import random
mx.random.seed(1)
ctx = mx.gpu()

ModuleNotFoundError: ignored

In [0]:
def preprocess(corpus, wtoi = {}, itow = []):
    tokenizer = nltk.tokenize.RegexpTokenizer(r'[\w\']+')
    sequence = []
    flag = 1
    if len(itow) == 0 :
        wtoi['<eos>']=len(itow)
        itow.append('<eos>')
        flag = 0
    for line in corpus.split('\n'):
        words = tokenizer.tokenize(line.lower())
        for word in words:
            if word in wtoi.keys():
                pass
            elif flag == 0:
                wtoi[word] = len(itow)
                itow.append(word)
            else:
                word = '<oov>'
            sequence.append(wtoi[word])
        sequence.append(wtoi['<eos>'])
    if flag == 0 :
        wtoi['<oov>']=len(itow)
        itow.append('<oov>')
    return np.array(sequence), wtoi, itow

In [4]:
with open("train.txt") as f:
    traincorpus = f.read()
print(traincorpus[:500])
with open("test.txt") as f:
    testcorpus = f.read()
print(testcorpus[:500])
testlines = testcorpus.split('\n')

The third was being run by the head of an investment firm
You wonder if he was manipulating the market with his bombing targets
Guerrillas killed an engineer , Asi Ali , from Tikrit
His mother was also killed in the attack
The Commission said it had no legal grounds for such an exclusion
This item is a small one and easily missed
But in my view it is highly significant
The situation in Iraq is only going to get better this way
If someone committed a crime against humanity , prosecute the person

From the AP comes this story
A la guerre c'est comme a la guerre
The US troops fired into the hostile crowd , killing 4
Read the entire article ; there 's a punchline , too
In Ramadi , there was a big demonstration
Now , people wonder if Google can even survive
That 's overstating it , I know
When they saw a cartoon of their prophet , people had to die
The battles and demonstrations were provoked by the US assault on Fallujah
The hottest item on Christmas wish lists this year is nuclear weapons

In [0]:
sequence, wtoi, itow = preprocess(traincorpus)

In [6]:
print(len(wtoi.keys()))
print(len(itow))
print(' '.join(traincorpus.split()[:500]))
print([itow[i] for i in sequence[:500]])

5896
5896
The third was being run by the head of an investment firm You wonder if he was manipulating the market with his bombing targets Guerrillas killed an engineer , Asi Ali , from Tikrit His mother was also killed in the attack The Commission said it had no legal grounds for such an exclusion This item is a small one and easily missed But in my view it is highly significant The situation in Iraq is only going to get better this way If someone committed a crime against humanity , prosecute the person A team from the court managed to take his deposition before he died The trial begins again Nov. 28 There has been talk that the night curfew might be implemented again Over 300 Iraqis are reported dead and 500 reported wounded in Fallujah alone Al - Iraqiyah tv said that ICDC were controlling Ramadi Now you can tour Chernobyl and write your own story This is not a post about fault - finding or assigning blame There is a lot to learn about Chernobyl There are a wealth of references on C

In [7]:
character_list = itow
vocab_size = len(itow)
print(character_list)
print("Length of vocab: %s" % vocab_size)

['<eos>', 'the', 'third', 'was', 'being', 'run', 'by', 'head', 'of', 'an', 'investment', 'firm', 'you', 'wonder', 'if', 'he', 'manipulating', 'market', 'with', 'his', 'bombing', 'targets', 'guerrillas', 'killed', 'engineer', 'asi', 'ali', 'from', 'tikrit', 'mother', 'also', 'in', 'attack', 'commission', 'said', 'it', 'had', 'no', 'legal', 'grounds', 'for', 'such', 'exclusion', 'this', 'item', 'is', 'a', 'small', 'one', 'and', 'easily', 'missed', 'but', 'my', 'view', 'highly', 'significant', 'situation', 'iraq', 'only', 'going', 'to', 'get', 'better', 'way', 'someone', 'committed', 'crime', 'against', 'humanity', 'prosecute', 'person', 'team', 'court', 'managed', 'take', 'deposition', 'before', 'died', 'trial', 'begins', 'again', 'nov', '28', 'there', 'has', 'been', 'talk', 'that', 'night', 'curfew', 'might', 'be', 'implemented', 'over', '300', 'iraqis', 'are', 'reported', 'dead', '500', 'wounded', 'fallujah', 'alone', 'al', 'iraqiyah', 'tv', 'icdc', 'were', 'controlling', 'ramadi', 'no

In [8]:
character_dict = wtoi
print(character_dict)

{'<eos>': 0, 'the': 1, 'third': 2, 'was': 3, 'being': 4, 'run': 5, 'by': 6, 'head': 7, 'of': 8, 'an': 9, 'investment': 10, 'firm': 11, 'you': 12, 'wonder': 13, 'if': 14, 'he': 15, 'manipulating': 16, 'market': 17, 'with': 18, 'his': 19, 'bombing': 20, 'targets': 21, 'guerrillas': 22, 'killed': 23, 'engineer': 24, 'asi': 25, 'ali': 26, 'from': 27, 'tikrit': 28, 'mother': 29, 'also': 30, 'in': 31, 'attack': 32, 'commission': 33, 'said': 34, 'it': 35, 'had': 36, 'no': 37, 'legal': 38, 'grounds': 39, 'for': 40, 'such': 41, 'exclusion': 42, 'this': 43, 'item': 44, 'is': 45, 'a': 46, 'small': 47, 'one': 48, 'and': 49, 'easily': 50, 'missed': 51, 'but': 52, 'my': 53, 'view': 54, 'highly': 55, 'significant': 56, 'situation': 57, 'iraq': 58, 'only': 59, 'going': 60, 'to': 61, 'get': 62, 'better': 63, 'way': 64, 'someone': 65, 'committed': 66, 'crime': 67, 'against': 68, 'humanity': 69, 'prosecute': 70, 'person': 71, 'team': 72, 'court': 73, 'managed': 74, 'take': 75, 'deposition': 76, 'before':

In [0]:
time_numerical = sequence

In [10]:
#########################
#  Check that the length is right
#########################
print(len(time_numerical))

#########################
#  Check that the format looks right
#########################
print(time_numerical[:20])

#########################
#  Convert back to text
#########################
print(" ".join([character_list[idx] for idx in time_numerical[:39]]))

36408
[ 1  2  3  4  5  6  1  7  8  9 10 11  0 12 13 14 15  3 16  1]
the third was being run by the head of an investment firm <eos> you wonder if he was manipulating the market with his bombing targets <eos> guerrillas killed an engineer asi ali from tikrit <eos> his mother was also


In [0]:
def one_hots(numerical_list, vocab_size=vocab_size):
    result = nd.zeros((len(numerical_list), vocab_size), ctx=ctx)
    for i, idx in enumerate(numerical_list):
        result[i, int(idx)] = 1.0
    return result

In [12]:
print(one_hots(time_numerical[:2]))


[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]]
<NDArray 2x5896 @gpu(0)>


In [0]:
def textify(embedding):
    indices = nd.argmax(embedding, axis=1).asnumpy()
    #result = " ".join([character_list[int(idx)] if int(idx) != 0 else '\n' for idx in indices])
    result = (" ".join([character_list[int(idx)] for idx in indices])+' ').replace('<eos> ','\n')
    return result

In [14]:
print(textify(one_hots(time_numerical[:40])))
print('\n\n')
print(textify(one_hots(time_numerical[-40:])))

the third was being run by the head of an investment firm 
you wonder if he was manipulating the market with his bombing targets 
guerrillas killed an engineer asi ali from tikrit 
his mother was also killed 



in on a sunday to replace a shredded tire 
ok one bad experience fine 
once again i waited for quite a bit before being attended to 
so i got just my other rear tire replaced 




In [15]:
seq_length = 64
# -1 here so we have enough characters for labels later
num_samples = (len(time_numerical) - 1) // seq_length
dataset = one_hots(time_numerical[:seq_length*num_samples]).reshape((num_samples, seq_length, vocab_size))
print(textify(dataset[0]))

the third was being run by the head of an investment firm 
you wonder if he was manipulating the market with his bombing targets 
guerrillas killed an engineer asi ali from tikrit 
his mother was also killed in the attack 
the commission said it had no legal grounds for such an exclusion 
this item is a small one and 


In [0]:
batch_size = 32

In [17]:
print('# of sequences in dataset: ', len(dataset))
num_batches = len(dataset) // batch_size
print('# of batches: ', num_batches)
train_data = dataset[:num_batches*batch_size].reshape((batch_size, num_batches, seq_length, vocab_size))
# swap batch_size and seq_length axis to make later access easier
train_data = nd.swapaxes(train_data, 0, 1)
train_data = nd.swapaxes(train_data, 1, 2)
print('Shape of data set: ', train_data.shape)

# of sequences in dataset:  568
# of batches:  17
Shape of data set:  (17, 64, 32, 5896)


In [18]:
for i in range(3):
    print("***Batch %s:***\n %s \n %s \n\n" % (i, textify(train_data[i, :, 0]), textify(train_data[i, :, 1])))

***Batch 0:***
 the third was being run by the head of an investment firm 
you wonder if he was manipulating the market with his bombing targets 
guerrillas killed an engineer asi ali from tikrit 
his mother was also killed in the attack 
the commission said it had no legal grounds for such an exclusion 
this item is a small one and  
 intelligence 's lackluster performance in dealing with the ltte is startling 
six crew members were from the arakan region of myanmar 
the boat was heading toward cox 's bazar in bangladesh 
subsequently phuket became the ltte 's main backup base 
he also writes for aakrosh india 's defense tied quarterly journal 
both india and afghanistan deny the claim 
 


***Batch 1:***
 easily missed 
but in my view it is highly significant 
the situation in iraq is only going to get better this way 
if someone committed a crime against humanity prosecute the person 
a team from the court managed to take his deposition before he died 
the trial begins again nov 28 

In [19]:
labels = one_hots(time_numerical[1:seq_length*num_samples+1])
train_label = labels.reshape((batch_size, num_batches, seq_length, vocab_size))
train_label = nd.swapaxes(train_label, 0, 1)
train_label = nd.swapaxes(train_label, 1, 2)
print(train_label.shape)

(17, 64, 32, 5896)


In [20]:
print(textify(train_data[10, :, 3]))
print(textify(train_label[10, :, 3]))

return has no income assets or liabilities 
despite the name this entity appears to be a mtm company 
80y enron broadband acquisition inc 
i think this entity should stay with 83n 
so i suggest it be assigned to leon 
it should be 4 easy 
1579 ebs network co division of 17h 
this one should possibly be assigned 
has no income assets or liabilities 
despite the name this entity appears to be a mtm company 
80y enron broadband acquisition inc 
i think this entity should stay with 83n 
so i suggest it be assigned to leon 
it should be 4 easy 
1579 ebs network co division of 17h 
this one should possibly be assigned to 


In [0]:
num_inputs = vocab_size
num_hidden = 256
num_outputs = vocab_size

########################
#  Weights connecting the inputs to the hidden layer
########################
Wxg = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxi = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxf = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxo = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01

########################
#  Recurrent weights connecting the hidden layer across time steps
########################
Whg = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whi = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whf = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Who = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01

########################
#  Bias vector for hidden layer
########################
bg = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bi = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bf = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bo = nd.random_normal(shape=num_hidden, ctx=ctx) * .01

########################
# Weights to the output nodes
########################
Why = nd.random_normal(shape=(num_hidden,num_outputs), ctx=ctx) * .01
by = nd.random_normal(shape=num_outputs, ctx=ctx) * .01

In [0]:
params = [Wxg, Wxi, Wxf, Wxo, Whg, Whi, Whf, Who, bg, bi, bf, bo, Why, by]

for param in params:
    param.attach_grad()


In [0]:
def softmax(y_linear, temperature=1.0):
    lin = (y_linear-nd.max(y_linear)) / temperature
    exp = nd.exp(lin)
    partition = nd.sum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition

In [24]:
####################
# With a temperature of 1 (always 1 during training), we get back some set of probabilities
####################
softmax(nd.array([[1, -1], [-1, 1]]), temperature=1.0)


[[ 0.88079703  0.11920292]
 [ 0.11920292  0.88079703]]
<NDArray 2x2 @cpu(0)>

In [25]:
####################
# If we set a high temperature, we can get more entropic (*noisier*) probabilities
####################
softmax(nd.array([[1,-1],[-1,1]]), temperature=1000.0)


[[ 0.50049996  0.49949998]
 [ 0.49949998  0.50049996]]
<NDArray 2x2 @cpu(0)>

In [26]:
####################
# Often we want to sample with low temperatures to produce sharp probabilities
####################
softmax(nd.array([[10,-10],[-10,10]]), temperature=.1)


[[ 1.  0.]
 [ 0.  1.]]
<NDArray 2x2 @cpu(0)>

In [0]:
def lstm_rnn(inputs, h, c, temperature=1.0):
    outputs = []
    for X in inputs:
        g = nd.tanh(nd.dot(X, Wxg) + nd.dot(h, Whg) + bg)
        i = nd.sigmoid(nd.dot(X, Wxi) + nd.dot(h, Whi) + bi)
        f = nd.sigmoid(nd.dot(X, Wxf) + nd.dot(h, Whf) + bf)
        o = nd.sigmoid(nd.dot(X, Wxo) + nd.dot(h, Who) + bo)
        #######################
        #
        #######################
        c = f * c + i * g
        h = o * nd.tanh(c)
        #######################
        #
        #######################
        yhat_linear = nd.dot(h, Why) + by
        yhat = softmax(yhat_linear, temperature=temperature)
        outputs.append(yhat)
    return (outputs, h, c)

In [0]:
# def cross_entropy(yhat, y):
#     return - nd.sum(y * nd.log(yhat))

def cross_entropy(yhat, y):
    return - nd.mean(nd.sum(y * nd.log(yhat), axis=0, exclude=True))

In [29]:
cross_entropy(nd.array([[.2,.5,.3], [.2,.5,.3]]), nd.array([[1.,0,0], [0, 1.,0]]))


[ 1.15129256]
<NDArray 1 @cpu(0)>

In [0]:
def average_ce_loss(outputs, labels):
    assert(len(outputs) == len(labels))
    total_loss = 0.
    for (output, label) in zip(outputs,labels):
        total_loss = total_loss + cross_entropy(output, label)
    return total_loss / len(outputs)

In [0]:
def SGD(params, lr):    
    for param in params:
        param[:] = param - lr * param.grad

In [0]:
def testaccuracy(mode, temperature=1.0):
    #####################################
    # Initialize the string that we'll return to the supplied prefix
    #####################################
    global testlines
    total = 0
    correct = 0
    for prefix in testlines :
        prefix_numerical, _, __ = preprocess(prefix, wtoi, itow)

        #####################################
        # Prepare the prefix as a sequence of one-hots for ingestion by RNN
        #####################################
        if mode == 0 :
            string_numerical = list(prefix_numerical[:-1])
            num_chars = 1
        else :
            string_numerical = list(prefix_numerical[:(len(prefix_numerical)+1)//2])
            num_chars = len(prefix_numerical) - len(string_numerical)

        input_sequence = one_hots(string_numerical)

        h = nd.zeros(shape=(1, num_hidden), ctx=ctx)
        c = nd.zeros(shape=(1, num_hidden), ctx=ctx)

        for i in range(num_chars):
            outputs, h, c = lstm_rnn(input_sequence, h, c, temperature=temperature)
            #choice = np.random.choice(vocab_size, p=outputs[-1][0].asnumpy())
            choice = np.argmax((outputs[-1][0]).asnumpy(),axis=0)
            #print(choice)
            string_numerical.append(choice)
            if choice == prefix_numerical[len(string_numerical)-1]:
                correct += 1
            total += 1
            input_sequence = one_hots([choice])
        #print(string_numerical)
    acc = float(correct)/total
    #return textify(one_hots(string_numerical))
    return acc

In [33]:
print(testaccuracy(0, temperature=.1))

0.0


In [0]:
epochs = 2000
moving_loss = 0.

learning_rate = 2.0

# state = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
for e in range(epochs):
    ############################
    # Attenuate the learning rate by a factor of 2 every 100 epochs.
    ############################
    if ((e+1) % 100 == 0):
        learning_rate = learning_rate / 2.0
    h = nd.zeros(shape=(batch_size, num_hidden), ctx=mx.gpu())
    c = nd.zeros(shape=(batch_size, num_hidden), ctx=mx.gpu())
    for i in range(num_batches):
        data_one_hot = train_data[i]
        label_one_hot = train_label[i]
        with autograd.record():
            outputs, h, c = lstm_rnn(data_one_hot, h, c)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = nd.mean(loss).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(loss).asscalar()

    print("Epoch %s. Loss: %s, Test Accuracy: %f" % (e, moving_loss, 100*testaccuracy(0, temperature=.1)))

Epoch 0. Loss: 8.64264800933, Test Accuracy: 100.000000
Epoch 1. Loss: 8.50281664915, Test Accuracy: 0.000000
Epoch 2. Loss: 8.32879803211, Test Accuracy: 99.707602
Epoch 3. Loss: 8.11411426274, Test Accuracy: 100.000000
Epoch 4. Loss: 7.90261438226, Test Accuracy: 0.000000
Epoch 5. Loss: 7.71149917558, Test Accuracy: 0.000000
Epoch 6. Loss: 7.55044609956, Test Accuracy: 0.000000
Epoch 7. Loss: 7.40931205458, Test Accuracy: 100.000000
Epoch 8. Loss: 7.2854081501, Test Accuracy: 0.146199
Epoch 9. Loss: 7.17910374921, Test Accuracy: 100.000000
Epoch 10. Loss: 7.08468651579, Test Accuracy: 100.000000
Epoch 11. Loss: 7.00404844681, Test Accuracy: 100.000000
Epoch 12. Loss: 6.93300325692, Test Accuracy: 100.000000
Epoch 13. Loss: 6.87174871282, Test Accuracy: 97.953216
Epoch 14. Loss: 6.81894663819, Test Accuracy: 100.000000
Epoch 15. Loss: 6.77446821615, Test Accuracy: 100.000000
Epoch 16. Loss: 6.73535949881, Test Accuracy: 100.000000
Epoch 17. Loss: 6.70142696194, Test Accuracy: 0.000000

In [0]:
print("Task a. After training with one hot encoding : \n")
print("")
print("Epoch %s. Loss: %s Test Accuracy: %f" % (e, moving_loss, testaccuracy(0, temperature=.1)))